<h1>Multi-layer Perceptron - MNIST</h1>

Creiamo una rete MLP per riconoscere le 10 cifre numeriche scritte a mano. Per fare questo usiamo un dataset già presente in <strong>scikit-learn</strong>: MNIST database (<em>Modified National Institute of Standards and Technology database</em>). Questo database contiene una vasta base di dati di cifre scritte a mano da studenti delle scuole superiori e da impiegati dell’Ufficio Censimento degli USA.

In [ ]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784')

print("Campi del dataset\n",mnist.keys())
print(mnist['DESCR'])

Dopo aver letto la descrizione capiamo che i primi 60 000 campioni vengono proposti per il training set mentre i successivi 10 000 per il test set. I dati sono memorizzati come immagini 28x28 in scala di grigio, pertanto ogni cifra è rappresentata come un array di 784 numeri interi compresi tra 0 e 255 per rappresentare le gradazioni di grigio (nero=0, bianco=255).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

y = mnist.target
X = mnist.data
print(X.shape) # 70000 campioni di 784 pixel = 28x28 in gradazioni di grigio
X.head()

In [ ]:
# disegnamo un campione
plt.imshow(X.loc[0].values.reshape(28, 28), cmap='gray')
plt.title(f"Label: {y[0]}") 
plt.axis("off")  # rimuove gli assi
plt.show()

In [ ]:
X = X / 255   # normalizziamo i dati ottenendo valori tutti compresi tra 0 e 1 (opzionale)

In [ ]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:],y[:60000], y[60000:]

Passiamo ad addestrare la rete scegliendo un solo livello nascosto con 50 nodi. Utilizziamo il parametro <kbd>verbose = True</kbd> per vedere l’andamento dell’apprendimento.

In [ ]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(50,), max_iter=20, verbose=True, random_state=1, learning_rate_init=0.1)
mlp.fit(X_train, y_train)

print("Training set score: %f" % mlp.score(X_train, y_train))
print("Test set score: %f" % mlp.score(X_test, y_test))

Notiamo dall’output che il processo termina prima delle 20 epoche perché c’è un peggioramento per 10 epoche consecutive, a partire dall’iterazione numero 6. Infatti se guardiamo l’andamento dell’errore, osserviamo che esso raggiunge il suo minimo dopo 5 epoche. Siccome ci interessa conoscere l’accuratezza, fermiamo il processo in questo punto e vediamo che valore essa assume.

In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(50,), max_iter=5, verbose=True, random_state=1, learning_rate_init=0.1)
mlp.fit(X_train, y_train)

print("Training set score: %f" % mlp.score(X_train, y_train))
print("Test set score: %f" % mlp.score(X_test, y_test))

Dall’output capiamo che sebbene l’algoritmo non converga, l’accuratezza è leggermente migliorata rispetto a quella ottenuta prima, dopo 16 iterazioni. Fino a ora abbiamo usato l’algoritmo di ottimizzazione di default <b>adam</b>. Adesso usiamone un altro: l’<b>sgd</b>, cioè lo <i>stocastic gradient descent</i>. La libreria <strong>scikit-learn</strong> prevede anche un terzo algoritmo di training l’L-BFGS, la cui trattazione però esula dallo scopo di questo testo. Qui di seguito troviamo il codice in cui usiamo sgd e il corrispondente output.

In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(50,), max_iter=20, verbose=True, random_state=1, learning_rate_init=.1, solver = "sgd")
mlp.fit(X_train, y_train)

print("Training set score: %f" % mlp.score(X_train, y_train))
print("Test set score: %f" % mlp.score(X_test, y_test))

Dall’output capiamo che è andata decisamente meglio! Inoltre osserviamo che l’errore sta continuando a scendere. Questo vuol dire che stiamo rischiando l’overfitting? Per accertarcene impostiamo un numero diverso di epoche e controlliamo di volta in volta l’accuratezza.

In [ ]:
start = 3  # numero iniziale di epoche
stop = 30  # numero finale di epoche 
passo = 2  # passo incrementale

vEpochs = np.arange(start, stop+1, passo)
yTrain = []  #lista dell'accuratezza sui dati di train per disegnare un grafico
yTest = []   #lista dell'accuratezza sui dati di test per disegnare un grafico

for e in vEpochs:
    mlp = MLPClassifier(hidden_layer_sizes=(50,), max_iter=e, random_state=1, learning_rate_init=.1, solver = "sgd") 
    mlp.fit(X_train, y_train)
    print(e)
    accTrain = mlp.score(X_train, y_train)
    print("Training set score: %f" % accTrain)
    yTrain.append(accTrain)
    accTest = mlp.score(X_test, y_test)
    print("Test set score: %f" % accTest)
    yTest.append(accTest)

Creiamo il grafico corrispondente.

In [ ]:
#grafico dell'accuratezza 
plt.plot(vEpochs, yTrain, c='b', label = 'Train' )
plt.plot(vEpochs, yTest, c='g', label = 'Test')
plt.xlabel("epoche", fontsize=15)
plt.ylabel("accuratezza", fontsize=15)
plt.tick_params(axis='both', labelsize='15')
plt.legend(fontsize=12)

Sia dagli output sia dal grafico notiamo che il numero di epoche ottimale è 21, perché proseguendo con l’addestramento si cade nell’overfitting: le performance sul set di training migliorano, mentre quelle sul set di test peggiorano.

In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(50,), max_iter=21, random_state=1, learning_rate_init=.1, solver = "sgd") 
mlp.fit(X_train, y_train)

Proviamo a testare il nostro modello

In [ ]:
import cv2

# carica l'immagine in scala di grigi e ridimensionala a 28x28
image = cv2.imread("3.png", cv2.IMREAD_GRAYSCALE)
image = cv2.resize(image, (28, 28))
# normalizza i valori
image = image / 255 
# converte l'immagine in un vettore 1x784
x_sample = image.flatten().reshape(1, -1)

In [ ]:
'''
from PIL import Image

image = Image.open("3.png").convert("L")  # "L" = grayscale
image = image.resize((28, 28))
image_array = np.array(image)  
image_array = image_array / 255 
x_sample = image_array.flatten().reshape(1, -1)
'''

In [ ]:
y_sample = mlp.predict(x_sample)
print(y_sample)